# Chapter 12 C and C++

from Gayle Laakmann McDowell's "Cracking the Coding Interview", 6th ed.

Ron Wu 

## 12.1 print the last k lines of an input file

In [1]:
import ROOT
ROOT.toCpp()

Welcome to ROOTaaS 6.06/08


Notebook is in Cpp mode


In [2]:
.cpp -d
using namespace std;

void return_k_lines(int k) {
    int totline = 0;
    ifstream input( "data/testLine.txt" );
    for( string line; getline( input, line ); )
    {
        ++totline;
    }
    totline -= k; 
    
    ifstream input2( "data/testLine.txt" );
    for( string line; getline( input2, line ); )
    { 
        --totline;
        if (totline < 0) {
            cout<<line<<endl;
        } 
    } 
}

In [3]:
int k = 3;
return_k_lines(k);

line i
line j
the end


## 12.2 reserse a null-ended string

In [4]:
.cpp -d
void reserveString(char str[]) {
    cout<<str<<endl;
    int count = 0;
    char s = str[0];
    while (s != '\0'){
        count++;
        s = str[count];
    }
    for (int i = 0; i < count/2; i++){
        
        char temp = str[i];
        str[i] = str[count-i-1];
        str[count-i-1] = temp;
    }
}


In [5]:
char str[] = "abcdefg";
reserveString(str); 
cout <<str;

abcdefg
gfedcba

## 12.3 implement a hash table and a STL map

In [6]:
//unordered_map implements a usual hash table, giving O(1) retrieving and O(1) insertion
unordered_map<int, string> unord_m = {{1, "one"}, {2, "two"}};

//STL map is kind like a hash table, and the keys are constructed by binary search tree
//giving O(logn) retrieving and insertion
 
map<int, string> ord_m = {{1, "one"}, {2, "two"}};
ord_m.insert({0, "zero"});

for(auto i=ord_m.begin(); i!=ord_m.end(); ++i) {
    cout << (*i).first << ": " << (*i).second << endl;
} 

0: zero
1: one
2: two


In [7]:
//for collisions, use multimap
multimap<int, string> ord_multi_m = {{2, "two"}, {1, "one"}, {2, "another two"}};

for( auto i=ord_multi_m.begin(); i!=ord_multi_m.end(); ++i)
{
    cout << (*i).first << ": " << (*i).second << endl;
}
cout<<"multimap index is hard"<<endl;
auto ord_multi_m_2 = ord_multi_m.equal_range(2);
for( auto i=ord_multi_m_2.first; i!=ord_multi_m_2.second; ++i)
{
    cout << (*i).first << ": " << (*i).second << endl;
}

1: one
2: two
2: another two
multimap index is hard
2: two
2: another two


## 12.4 what is virtual function?

In [8]:
//virtual function makes extension or plug-in possible.
//more specifically, using dynamic binding, derived classes can modify behavior of
//functions that were originally built for the base classes

In [9]:
class Animal {
public: 
    virtual void speak(){
        cout<<"I am an animal!"<<endl;
    }
    virtual ~Animal(){
        cout<<"animal is gone"<<endl;
    }
};

In [10]:
class Dog : public Animal{

    string name = "goofy";
    
public:
    void speak(){
        cout<<"I am an dog!"<<endl;
        cout<<"My name is "<<name<<endl;
    }
    void barking(){
        cout<<"I am sparky!"<<endl;
    }
    
    ~Dog(){
        cout<<"dog is gone"<<endl;
    }
};

In [11]:
.cpp -d
void animal_speak(Animal *a) {
    a->speak();
}

In [12]:
Animal *a = new Animal();
animal_speak(a);

Dog *d = new Dog();
animal_speak(d);  

delete a;
delete d;

I am an animal!
I am an dog!
My name is goofy
animal is gone
dog is gone
animal is gone


## 12.5 shallow vs deep copy

In [13]:
//shallow copies member values. if some members are reference objects,
//then shallow copy will just copy the pointers
//deep copies everything in every level

## 12.6 what is volatile keyword?

In [14]:
//prevents the compiler from applying any optimizations on the code 
//and ensures values of variables gets reload every time

## 12.7 why do we apply virtual to the base destructor?

In [15]:
Animal *a1 = new Dog(); //without it, only the Animal destructor will be called.

animal_speak(a1);
delete a1;

//if we did the normal thing
//Dog *d1 = new Dog(); 
//then Animal destructor would be called without making it virtual.

I am an dog!
My name is goofy
dog is gone
animal is gone


## 12.8 copy node

In [16]:
struct Node{
    Node *left, *right;
    int * data;
    Node(int in){
        data = new int; 
        //better to use malloc for primitive have no user defined constructors,destrcutors
        //data = (int *)malloc(sizeof(int *));
        
        (*data) = in;
        left = NULL;
        right = NULL;
    }
    ~Node(){
        delete data; 
        //then use free 
        //free(data);
    }
};

In [17]:
.cpp -d
Node * copy_nodes(Node * node){
    Node * node_copy = new Node(*(node->data));
    node_copy->left = node->left;
    node_copy->right = node->right;
    return node_copy;
}

In [18]:
Node *node_1 = new Node(1);
Node *node_2 = new Node(2);
Node *node_3 = new Node(3);

node_2->left = node_1;
node_2->right = node_3;

Node *node_2_copy = copy_nodes(node_2);

cout<<"copied value is "<<*node_2_copy->data<<endl;
cout<<"copied left value is "<<*node_2_copy->left->data<<endl;
cout<<"copied right value is "<<*node_2_copy->right->data<<endl;

delete node_1;
delete node_2;
delete node_3;
delete node_2_copy;

copied value is 2
copied left value is 1
copied right value is 3


## 12.9 smart pointer with templates

In [19]:
template <class T>
class smartpointer{
    
    int count;
    T * data;
    
public:
    smartpointer(T * src){
        data = src;
        count = 1;
    }
    //assignment operator overload
    smartpointer<T> & operator = (smartpointer<T> *src){
        if (src == NULL){  //set NULL
            count = 0;
            delete data;
            data = NULL;
        }
        else{
            if (this != src){
                delete data;
                data = src->data;
            }
        }
        return *this;
    }
    
    ~smartpointer(){
        count = 0;
        delete data;
        data = NULL;
    }
};

class Person{
    string name; 
public:
    Person(string n){
        name = n; 
    }
    ~Person(){
        cout<<name<<" left"<<endl; 
    }
};


In [20]:
//normally we do this
Person * p = new Person("harry potter");
delete p; 
p = NULL;

//now we have smartpointer, only make NULL and delete will be called

Person * p_1 = new Person("ron weasley");
smartpointer<Person> sp(p_1);
sp = NULL;

harry potter left
ron weasley left


## 12.10 malloc

In [21]:
.cpp -d
void aligned_free(void *  p){
    free(p);
}

void * align_malloc(size_t size_of_class,unsigned num_class_block){
    return malloc(size_of_class*num_class_block);
}

In [22]:
void * ptr = align_malloc(1000,128); //void pointer points to any memory size
aligned_free(ptr);

## 12.11 2D alloc

In [23]:
.cpp -d
int ** my2DAlloc(int row_num, int col_num){
    auto row = (int**)malloc(row_num*sizeof(int*));
    for (int i = 0; i < row_num; ++i){
        row[i]= (int*)malloc(col_num*sizeof(int));
    }
    return row;
}

void my2DDealloc(int ** row, int row_num){
    for (int i = 0; i < row_num; ++i){
        free(row[i]);
    }
    free(row);
}

In [24]:
int row_num = 4;
int col_num = 5;
auto mat2D = my2DAlloc(row_num,col_num);
my2DDealloc(mat2D, row_num);